In [49]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [3]:
df1_i = pd.read_csv('raw_account_70_new.csv')
df1_t = pd.read_csv('raw_account_30_new.csv')
df2_i = pd.read_csv('raw_data_70_new.csv')
df2_t = pd.read_csv('raw_data_30_new.csv')
df3_i = pd.read_csv('raw_enquiry_70_new.csv')
df3_t = pd.read_csv('raw_enquiry_30_new.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12,20,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#removing customer number repitition
c_1 = df1_i['customer_no'] + 13000
c_2 = df2_i['customer_no'] + 13000
c_3 = df3_i['customer_no'] + 13000
df1_i['customer_no'] = c_1
df2_i['customer_no'] = c_2
df3_i['customer_no'] = c_3


len_1 = len(df1_i)
len_2 = len(df2_i)
len_3 = len(df3_i)
df1 = pd.concat([df1_i,df1_t],axis=0)
df2 = pd.concat([df2_i,df2_t],axis=0)
df3 = pd.concat([df3_i,df3_t],axis=0)

coll = [df1,df2,df3]
for i in coll:
    i.reset_index(inplace= True)
    i.drop(['index'],axis=1,inplace=True)

In [7]:
#prepocessing df1
col_dates_1 = ['dt_opened', 'upload_dt','opened_dt', 'last_paymt_dt', 'closed_dt',
             'reporting_dt','paymt_str_dt', 'paymt_end_dt']
for col in col_dates_1:
    df1[col] = pd.to_datetime(df1[col])

roi = []
for i in df1.rateofinterest:
    try:
        roi.append(float(i.replace('"','')))
    except:
        roi.append(i)
df1['rateofinterest'] = roi
df1['rateofinterest'] = df1.rateofinterest.astype('float')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265897 entries, 0 to 265896
Data columns (total 21 columns):
dt_opened              265897 non-null datetime64[ns]
customer_no            265897 non-null int64
upload_dt              265897 non-null datetime64[ns]
acct_type              265897 non-null int64
owner_indic            265897 non-null int64
opened_dt              265268 non-null datetime64[ns]
last_paymt_dt          229707 non-null datetime64[ns]
closed_dt              110179 non-null datetime64[ns]
reporting_dt           265897 non-null datetime64[ns]
high_credit_amt        253327 non-null float64
cur_balance_amt        265897 non-null int64
amt_past_due           1274 non-null float64
paymenthistory1        265897 non-null object
paymenthistory2        111774 non-null object
paymt_str_dt           265895 non-null datetime64[ns]
paymt_end_dt           265895 non-null datetime64[ns]
creditlimit            69344 non-null float64
cashlimit              50027 non-null float64
r

In [8]:
#for deciding if categorical or numerical
uni_df2 = [len(df2[i].unique()) for i in df2]
nulls_df2 = [df2[i].isnull().sum() for i in df2]
dt_df2 = [df2[i].dtype for i in df2]
analyz_df2 = pd.DataFrame([uni_df2,nulls_df2,dt_df2],index=['uniq','nulls','dty'],columns=df2.columns)
analyz_df2.T

,uniq,nulls,dty
dt_opened,197,0,object
customer_no,34136,0,int64
entry_time,304,24,object
feature_1,8,24,object
feature_2,289,4047,object
feature_3,272,4047,float64
feature_4,4,24,float64
feature_5,2,24,object
feature_6,2,24,float64
feature_7,513,24,float64


In [9]:
# email processing
df2.feature_24.fillna(df2.feature_24.mode()[0],inplace= True)
bb1 = [i[:5].upper() for i in df2.feature_24]
cc1 = [i.split('.')[-1].upper() for i in df2.feature_24]
bb2 = [i for i,_ in Counter(bb1).most_common(7)]
cc2 = [i for i,_ in Counter(cc1).most_common(6)]
bb = [] # contains first part of email
for i in bb1:
    if i in bb2:
        bb.append(i)
    else:
        bb.append('OTHER')
cc = [] # contains second part of email
for i in cc1:
    if i in cc2:
        cc.append(i)
    else:
        cc.append('OTHER')
df2['feature_24'] = cc # can also use bb

df2.feature_45.fillna(df2.feature_45.mode()[0],inplace= True)
bb1 = [i[:5].upper() for i in df2.feature_45]
cc1 = [i.split('.')[-1].upper() for i in df2.feature_45]
bb2 = [i for i,_ in Counter(bb1).most_common(7)]
cc2 = [i for i,_ in Counter(cc1).most_common(6)]
bb = [] # contains first part of email
for i in bb1:
    if i in bb2:
        bb.append(i)
    else:
        bb.append('OTHER')
cc = [] # contains second part of email
for i in cc1:
    if i in cc2:
        cc.append(i)
    else:
        cc.append('OTHER')
df2['feature_45'] = bb # can also use cc

In [10]:
df2['feature_29'] = df2.feature_29//10000 #pincodes 
df2['feature_66'] = df2.feature_66//10000
# used 44 & dropped 28,44,45 because all represent nearly same thing

In [11]:
#Dates
jan_2016 = pd.to_datetime('Jan-2016')  # Reference Date
li_op = []
for i in df2.dt_opened:
    li_op.append((jan_2016 - pd.to_datetime(i)).days)
df2['dt_opened'] = li_op

df2.entry_time.fillna(df2.entry_time.mode()[0],inplace=True)
li_et = []
for i in df2.entry_time:
    li_et.append((jan_2016 - pd.to_datetime(i)).days)
df2['entry_time'] = li_et

df2.feature_2.fillna(df2.feature_2.mode()[0],inplace=True)
li_2 = []
for i in df2.feature_2:
    li_2.append((jan_2016 - pd.to_datetime(i)).days)
df2['feature_2'] = li_2

df2.feature_21.fillna('1-Jan-86',inplace=True)
li_21 = []
for i in df2.feature_21:
    year_21 = int(i.split('-')[-1])
    if year_21<17:
        year_21 += 2000
    else:
        year_21 += 1900
    li_21.append(2016 - year_21)
df2['feature_21'] = li_21 

df2['feature_30'] = df2.feature_30.fillna(df2.feature_30.mode()[0])
df2['feature_30'] = 2016 - df2.feature_30

df2['feature_39'] = df2.feature_39.replace({0:2015})
df2['feature_39'] = df2.feature_39.fillna(df2.feature_39.mode()[0])
df2['feature_39'] = 2016 - df2.feature_39

df2.feature_53.fillna('16',inplace=True)
li_53 = []
for i in df2.feature_53:
    year_53 = int(i.split('-')[-1])
    if year_53<17:
        year_53 += 2000
    else:
        year_53 += 1900
    li_53.append(2016 - year_53)
df2['feature_53'] = li_53

df2.feature_63.fillna(df2.feature_63.mode()[0],inplace=True)
df2['feature_63'] = [2016 - int(i[:4]) for i in df2.feature_63]

df2.feature_70.replace({'0-0':'1-Jan'},inplace=True)
df2.feature_70.fillna('1-Jan',inplace=True)
li_70 = []
for i in df2.feature_70:
    if i[-2:] == '00':
        i = '1-' + i[:3] + '-2015'
    elif i[0] == '0':
        i = '1-' + i.split('-')[-1] + '-2015'
    else:
        i += '-2015'
    li_70.append((jan_2016 - pd.to_datetime(i)).days)
df2['feature_70'] = li_70

In [12]:
# designation
df2.feature_38.fillna('NAN',inplace=True)
aa = (' '.join(df2.feature_38.tolist())).split(' ') # all the designations
aa =  [i for i,_ in Counter(aa).most_common(100)] 
f_38 = pd.DataFrame()
for i in aa:
     f_38 = pd.concat([f_38,pd.Series([int(i in j.split(' ')) for j in df2.feature_38], name= 'desg_'+ i )],axis=1)

In [13]:
#categorical vars
# not using 24
cats = [1,8,11,12,14,19,23,25,27,29,32,33,34,36,37,45,46,48,51,55,56,57,58,59,60,62,66,67,72,73,79] 
cats = ['feature_' + str(i) for i in cats]
num_cats = [14,19,25,29,34,55,56,66,67] #categorical vars dtype float
num_cats = ['feature_' + str(i) for i in num_cats]
df2[num_cats] = df2[num_cats].astype('str')
df2_cats = pd.get_dummies(df2[cats])
df2_cats = pd.concat([df2_cats,f_38],axis=1)

In [15]:
#making suggested variables
rec_365 = []
for i in df3.enquiry_dt:
    try:
        rec_365.append(i[-2:] == '15')
    except:
        rec_365.append(False)
rdf3 = df3[rec_365]
count_enquiry_recency_365 = pd.Series([len(rdf3[rdf3.customer_no == i]) for i in df2.customer_no])

In [16]:
col_dates_3 = ['dt_opened', 'upload_dt', 'enquiry_dt']
for col in col_dates_3:
    df3[col] = pd.to_datetime(df3[col])

In [17]:
#making suggested variables
df1.paymenthistory1.fillna('',inplace=True)
df1.paymenthistory2.fillna('',inplace=True)
oct_2015 =  pd.to_datetime('1-Oct-2015')

rec_90 = df3[df3.enquiry_dt>= oct_2015]

count_enquiry_recency_90 = []
mean_diff_lastpaymt_opened_dt = []
payment_history_avg_dpd_0_29_bucket = []
total_diff_lastpaymt_opened_dt = []
utilisation_trend = []
Ratio_currbalance_creditlimit  = []
mean_diff_open_enquiry_dt = []
payment_history_mean_length = []
max_freq_enquiry = []

for cust in df2.customer_no:
    aa = df1[df1.customer_no == cust]
    zz = df3[df3.customer_no == cust]
    cc = []
    ee = []
#     for k in range(len(aa)):
    bb = aa['reporting_dt'] - aa['paymt_str_dt']
    dd = aa['last_paymt_dt'] - aa['opened_dt']
        
    bb = [i.days for i in bb]
    dd = [i.days for i in dd]
    bb = pd.Series(bb)
    total_diff_lastpaymt_opened_dt.append(sum(dd))
    mean_diff_lastpaymt_opened_dt.append(np.mean(dd))
    payment_history_avg_dpd_0_29_bucket.append(len(bb[bb<=30]))
    
    cc = []
    ee = []
    f_1 = []
#     for k in range(len(aa)):
    try:
        utilisation_trend.append(((aa.cur_balance_amt.sum())*(aa.creditlimit.mean() + aa.cashlimit.mean()))/((aa.creditlimit.sum())*(aa.cur_balance_amt.mean())))
    except:
        utilisation_trend.append(np.NaN)

    try:
        Ratio_currbalance_creditlimit.append((aa.cur_balance_amt.sum())/(aa.creditlimit.sum()))
    except:
        Ratio_currbalance_creditlimit.append(np.NaN)
    
    gg = zz.dt_opened - zz.enquiry_dt
    mean_diff_open_enquiry_dt.append(gg.mean().days)
    
    jj = np.mean([len(i) + len(j) for i,j in zip(aa.paymenthistory1,aa.paymenthistory2)])
    payment_history_mean_length.append(jj)
    
    max_freq_enquiry.append(Counter(zz.enq_purpose).most_common(1)[0][0])
    
    count_enquiry_recency_90.append(len(rec_90[rec_90.customer_no == cust]))

    
count_enquiry_recency_90 = pd.Series(count_enquiry_recency_90)
max_freq_enquiry = pd.get_dummies(max_freq_enquiry)
payment_history_mean_length = pd.Series(payment_history_mean_length)
mean_diff_open_enquiry_dt = pd.Series(mean_diff_open_enquiry_dt)
Ratio_currbalance_creditlimit = pd.Series(Ratio_currbalance_creditlimit)
Ratio_currbalance_creditlimit.replace({np.inf: 2500, -np.inf: -110}, inplace=True)

mean_diff_lastpaymt_opened_dt = pd.Series(mean_diff_lastpaymt_opened_dt)
payment_history_avg_dpd_0_29_bucket = pd.Series(payment_history_avg_dpd_0_29_bucket)
total_diff_lastpaymt_opened_dt = pd.Series(total_diff_lastpaymt_opened_dt)
utilisation_trend = pd.Series(utilisation_trend)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


In [19]:
label = df2.Bad_label
#dropping columns like phone number or columns with only single value or columns wit lots of NaN 
df2.drop(['customer_no','feature_5','feature_6','feature_9','feature_10','feature_13','feature_15','feature_16',
          'feature_17','feature_18','feature_20','feature_22','feature_24','feature_28',
          'feature_31','feature_38','feature_43','feature_44','feature_47','feature_49','feature_50',
          'feature_54','feature_61','feature_74','feature_75','feature_77','Bad_label'],axis=1 , inplace=True)

In [22]:
df2_num = pd.concat([df2.drop(cats,axis=1), count_enquiry_recency_90,max_freq_enquiry,payment_history_mean_length,
                     mean_diff_open_enquiry_dt,count_enquiry_recency_365,Ratio_currbalance_creditlimit,mean_diff_lastpaymt_opened_dt,
                     payment_history_avg_dpd_0_29_bucket,total_diff_lastpaymt_opened_dt, utilisation_trend],axis= 1)
df2_num.columns = range(len(df2_num.columns))

#removing obvious outliers
for col in df2_num:
    df2_num[col] = df2_num[col].mask(df2_num[col] - df2_num[col].mean() > 5*(df2_num[col].std()) , df2_num[col].mean() +  5*(df2_num[col].std()) )
df2_num.describe().round()
df2_num.fillna(df2_num.mean(),inplace=True)

In [23]:
df2_num.describe().round()

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
count,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,...,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0,34136.0
mean,106.0,133.0,126.0,723.0,2.0,118304.0,35.0,1.0,15.0,31787.0,...,0.0,0.0,58.0,926.0,3.0,410.0,596.0,6.0,3237.0,5.0
std,69.0,69.0,68.0,35.0,1.0,72863.0,7.0,1.0,12.0,42790.0,...,0.0,0.0,22.0,570.0,4.0,918.0,275.0,4.0,2544.0,3.0
min,1.0,8.0,8.0,-1.0,1.0,0.0,19.0,0.0,1.0,1.0,...,0.0,0.0,9.0,8.0,0.0,-110.0,1.0,0.0,1.0,1.0
25%,46.0,76.0,74.0,701.0,1.0,73000.0,30.0,0.0,5.0,1.0,...,0.0,0.0,41.0,488.0,1.0,1.0,491.0,3.0,1997.0,3.0
50%,95.0,128.0,112.0,723.0,3.0,104000.0,33.0,0.0,11.0,30000.0,...,0.0,0.0,57.0,782.0,2.0,2.0,596.0,5.0,3237.0,5.0
75%,161.0,188.0,182.0,741.0,3.0,139000.0,38.0,2.0,24.0,43000.0,...,0.0,0.0,73.0,1262.0,5.0,11.0,596.0,8.0,3237.0,5.0
max,260.0,334.0,333.0,896.0,3.0,510589.0,72.0,6.0,52.0,640124.0,...,0.0,0.0,120.0,3530.0,22.0,2836.0,2541.0,28.0,22455.0,26.0


In [24]:
df2_num.describe().loc['min']

0       1.000000
1       8.000000
2       8.000000
3      -1.000000
4       1.000000
5       0.000000
6      19.000000
7       0.000000
8       1.000000
9       1.000000
10      1.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      1.000000
17      2.000000
18      1.000000
19      1.000000
20      1.000000
21     61.000000
22      2.000000
23      0.000000
24      1.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
30      0.000000
31      0.000000
32      0.000000
33      0.000000
34      0.000000
35      0.000000
36      0.000000
37      0.000000
38      0.000000
39      0.000000
40      0.000000
41      0.000000
42      0.000000
43      0.000000
44      0.000000
45      9.000000
46      8.000000
47      0.000000
48   -110.000000
49      1.000000
50      0.000000
51      1.000000
52      1.000007
Name: min, dtype: float64

In [52]:
mmsc = MinMaxScaler(feature_range=(0,1))
X = mmsc.fit_transform(df2_num)
X = np.concatenate([X,df2_cats.values],axis = 1)
X.shape

(34136, 479)

In [27]:
X_train = X[:len_2]
X_test = X[len_2:]
y_train = label.values[:len_2]
y_test = label.values[len_2:]

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV ,cross_val_score

In [29]:
clf = RandomForestClassifier()
param_grid = [{'n_estimators':[20,50,80],'max_features':[0.85,0.9,0.95,1.0],'max_depth':[None,2,5,6]}]
gsear = GridSearchCV(clf,param_grid, n_jobs = -1,cv = 4, scoring='roc_auc')

In [30]:
gsear.fit(X_train,y_train)

GridSearchCV(cv=4, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [20, 50, 80], 'max_features': [0.85, 0.9, 0.95, 1.0], 'max_depth': [None, 2, 5, 6]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [32]:
gsear.best_score_

0.6320337207869553

In [33]:
gsear.best_params_

{'max_depth': 2, 'max_features': 0.95, 'n_estimators': 80}

In [31]:
gini_coeff = 100*(2*gsear.best_score_ - 1)
print(gini_coeff)

26.40674415739106


In [36]:
gsear.best_estimator_.feature_importances_

array([0.        , 0.        , 0.00451282, 0.00682094, 0.        ,
       0.47156255, 0.00698243, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.04667333,
       0.00885015, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01163824, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02397702,
       0.        , 0.        , 0.        , 0.00681627, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01210544, 0.05854731, 0.08785711, 0.05775873, 0.04201434,
       0.00384225, 0.00746829, 0.00340829, 0.02404765, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [50]:
sum(gsear.best_estimator_.feature_importances_ > 0.00001)

41

In [53]:
X_p = PCA(n_components= 70).fit_transform(X)

In [65]:
clf2 = RandomForestClassifier(max_depth= None, n_estimators= 1000)
cross_val_score(clf2 , X_p , label,scoring='roc_auc',cv = 5)

KeyboardInterrupt: 

In [37]:
from keras import layers, models
from keras.callbacks import EarlyStopping

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [47]:
class_weight = {0: 1.,1: 23.3}
model1 = models.Sequential()
model1.add(layers.Dense(512,activation='relu',input_shape = (479,)))
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(512, activation='relu'))
model1.add(layers.Dropout(0.2))
model1.add(layers.Dense(128,activation= 'relu'))
model1.add(layers.Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='Adam', metrics =['acc'])

In [62]:
class_weight = {0: 1.,1: 23.3}
history = model1.fit( X,label.values ,epochs=100, validation_split=0.3,class_weight=class_weight)

Train on 23895 samples, validate on 10241 samples
Epoch 1/500
23895/23895 [==============================] - 7s 286us/step - loss: 0.2063 - acc: 0.9680 - val_loss: 4.3524 - val_acc: 0.8750
Epoch 2/500
23895/23895 [==============================] - 7s 286us/step - loss: 0.2042 - acc: 0.9682 - val_loss: 4.3732 - val_acc: 0.8896
Epoch 3/500
23895/23895 [==============================] - 7s 300us/step - loss: 0.2074 - acc: 0.9700 - val_loss: 5.0172 - val_acc: 0.8855
Epoch 4/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1807 - acc: 0.9697 - val_loss: 4.7332 - val_acc: 0.8900
Epoch 5/500
23895/23895 [==============================] - 7s 314us/step - loss: 0.1928 - acc: 0.9682 - val_loss: 4.5538 - val_acc: 0.8785
Epoch 6/500
23895/23895 [==============================] - 7s 308us/step - loss: 0.1888 - acc: 0.9694 - val_loss: 4.7283 - val_acc: 0.8755
Epoch 7/500
23895/23895 [==============================] - 7s 305us/step - loss: 0.2166 - acc: 0.9691 - val_loss: 3.

23895/23895 [==============================] - 7s 306us/step - loss: 0.1895 - acc: 0.9784 - val_loss: 4.6404 - val_acc: 0.8883
Epoch 60/500
23895/23895 [==============================] - 7s 303us/step - loss: 0.1727 - acc: 0.9806 - val_loss: 4.7585 - val_acc: 0.8875
Epoch 61/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1636 - acc: 0.9823 - val_loss: 5.2784 - val_acc: 0.8942
Epoch 62/500
23895/23895 [==============================] - 8s 315us/step - loss: 0.1683 - acc: 0.9791 - val_loss: 5.2035 - val_acc: 0.8964
Epoch 63/500
23895/23895 [==============================] - 7s 308us/step - loss: 0.1724 - acc: 0.9819 - val_loss: 5.2304 - val_acc: 0.8971
Epoch 64/500
23895/23895 [==============================] - 7s 307us/step - loss: 0.1754 - acc: 0.9804 - val_loss: 4.6974 - val_acc: 0.8860
Epoch 65/500
23895/23895 [==============================] - 7s 307us/step - loss: 0.1520 - acc: 0.9825 - val_loss: 4.8621 - val_acc: 0.8930
Epoch 66/500
23895/23895 [=======

23895/23895 [==============================] - 7s 308us/step - loss: 0.1478 - acc: 0.9876 - val_loss: 5.0395 - val_acc: 0.8986
Epoch 118/500
23895/23895 [==============================] - 7s 303us/step - loss: 0.1538 - acc: 0.9866 - val_loss: 4.9801 - val_acc: 0.8936
Epoch 119/500
23895/23895 [==============================] - 7s 307us/step - loss: 0.1419 - acc: 0.9882 - val_loss: 5.1432 - val_acc: 0.9010
Epoch 120/500
23895/23895 [==============================] - 7s 305us/step - loss: 0.1648 - acc: 0.9869 - val_loss: 4.6802 - val_acc: 0.8899
Epoch 121/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1740 - acc: 0.9877 - val_loss: 4.9008 - val_acc: 0.9014
Epoch 122/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1334 - acc: 0.9872 - val_loss: 5.3632 - val_acc: 0.9020
Epoch 123/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1733 - acc: 0.9858 - val_loss: 5.3221 - val_acc: 0.8970
Epoch 124/500
23895/23895 [

23895/23895 [==============================] - 7s 305us/step - loss: 0.1485 - acc: 0.9888 - val_loss: 5.4526 - val_acc: 0.9014
Epoch 176/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1724 - acc: 0.9878 - val_loss: 4.8576 - val_acc: 0.8911
Epoch 177/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1690 - acc: 0.9883 - val_loss: 5.0491 - val_acc: 0.8946
Epoch 178/500
23895/23895 [==============================] - 7s 303us/step - loss: 0.1384 - acc: 0.9890 - val_loss: 5.2741 - val_acc: 0.8972
Epoch 179/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1447 - acc: 0.9893 - val_loss: 5.4385 - val_acc: 0.8989
Epoch 180/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1365 - acc: 0.9901 - val_loss: 5.9358 - val_acc: 0.8993
Epoch 181/500
23895/23895 [==============================] - 7s 306us/step - loss: 0.1583 - acc: 0.9896 - val_loss: 5.3790 - val_acc: 0.8934
Epoch 182/500
23895/23895 [

23895/23895 [==============================] - 7s 304us/step - loss: 0.1986 - acc: 0.9898 - val_loss: 5.5221 - val_acc: 0.9017
Epoch 234/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1773 - acc: 0.9875 - val_loss: 5.4036 - val_acc: 0.9053
Epoch 235/500
23895/23895 [==============================] - 7s 303us/step - loss: 0.1643 - acc: 0.9891 - val_loss: 5.4987 - val_acc: 0.9032
Epoch 236/500
23895/23895 [==============================] - 7s 303us/step - loss: 0.1548 - acc: 0.9902 - val_loss: 5.5572 - val_acc: 0.9035
Epoch 237/500
23895/23895 [==============================] - 7s 304us/step - loss: 0.1445 - acc: 0.9918 - val_loss: 5.2932 - val_acc: 0.9020
Epoch 238/500
23895/23895 [==============================] - 7s 305us/step - loss: 0.1487 - acc: 0.9915 - val_loss: 5.3485 - val_acc: 0.9029
Epoch 239/500
23895/23895 [==============================] - 7s 305us/step - loss: 0.1236 - acc: 0.9916 - val_loss: 5.6529 - val_acc: 0.9046
Epoch 240/500
23895/23895 [

KeyboardInterrupt: 

{0: 1.0, 1: 23.3}

In [178]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))